In [1]:
#import libraries
import os
import cv2
import numpy as np
import keras
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam 


Helper functions and variables

In [3]:
def list_labels(file):
    labels_file = open(file, "r")
    labels = []
    
    for line in labels_file:
        label = line.strip()
        labels.append(label)
    
    labels_file.close()
    
    return labels


#list of all labels
class_names = list_labels("./CamSDD/Labels.txt")
class_name_labels = {class_name:i for i, class_name in enumerate(class_names)}



def load_data(folder):
    Category = ["training", "test", "validation"]
    output = []
    
    for category in Category:
        print("Loading {}".format(category))
        path = os.path.join(folder, category)
        print(path)
        images = []
        labels = []
        
        for sub_folder in os.listdir(path):
            label = class_name_labels[sub_folder]
            
            #Iterating through all images
            for file in os.listdir(os.path.join(path, sub_folder)):
                
                #getting the image path
                img_path = os.path.join(os.path.join(path, sub_folder), file)
                
                #appending image and corresponding label
                images.append(cv2.imread(img_path))
                labels.append(label)
            
        #check that data type doesn't affect accuracy
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')
        
        output.append((images, labels))
        
    return output



#displays 25 images with labels
def display_examples(class_names, images, labels):
    figsize = (20, 20)
    fig = plt.figure(figsize=figsize)
    fig.suptitle("Example of images", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # image = cv2.resize(images[i], figsize)
        plt.imshow(images[i].astype(np.uint8))
        plt.xlabel(class_names[labels[i]])
    plt.show()

Loading data

In [4]:
(train_images, train_labels), (test_images, test_labels), (validation_images, validation_labels)= load_data(".\\CamSDD")

Loading training
.\CamSDD\training
Loading test
.\CamSDD\test
Loading validation
.\CamSDD\validation


In [5]:
#shuffling train data
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

Visualize data

In [ ]:
display_examples(class_names, train_images, train_labels)

Creating model

In [6]:
#imports the mobilenet model and discards the last 1000 neuron layer.
#check how many neurons should be discarded
base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='sigmoid')(x)
x=Dropout(0.7)(x)
output = Dense(30, activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

In [ ]:
#checking model architecture
# print(model.summary())
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [7]:
# Freeze all layers except the ones we created above (check which method)
# for layer in model.layers[:-4]:
#     layer.trainable=False
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# check what loss function they used
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=1485,
#     decay_rate=0.1)

# model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='sparse_categorical_crossentropy')

class ADJUSTLR(keras.callbacks.Callback):
    def __init__ (self, model, freq, factor, verbose):
        self.model=model
        self.freq=freq
        self.factor =factor
        self.verbose=verbose
        self.adj_epoch=freq
    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 == self.adj_epoch: # adjust the learning rate
            lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
            new_lr=lr * self.factor
            self.adj_epoch +=self.freq
            if self.verbose == 1:
                print('\non epoch ',epoch + 1, ' lr was adjusted from ', lr, ' to ', new_lr)
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
freq=3
factor=0.1
verbose=1
callbacks=[ADJUSTLR(model, freq, factor, verbose)]
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(train_images, train_labels, batch_size=20, epochs=15)

Epoch 1/15
495/495 [==============================] - 328s 654ms/step - loss: 2.3049
Epoch 2/15
495/495 [==============================] - 295s 597ms/step - loss: 1.6388
Epoch 3/15
495/495 [==============================] - 299s 605ms/step - loss: 1.4537
Epoch 4/15
495/495 [==============================] - 294s 594ms/step - loss: 1.3266
Epoch 5/15
495/495 [==============================] - 296s 598ms/step - loss: 1.2448
Epoch 6/15
495/495 [==============================] - 294s 594ms/step - loss: 1.1896
Epoch 7/15
495/495 [==============================] - 295s 596ms/step - loss: 1.1109
Epoch 8/15
495/495 [==============================] - 298s 601ms/step - loss: 1.0867
Epoch 9/15
495/495 [==============================] - 317s 641ms/step - loss: 1.0431
Epoch 10/15
495/495 [==============================] - 302s 611ms/step - loss: 1.0134
Epoch 11/15
495/495 [==============================] - 292s 589ms/step - loss: 0.9811
Epoch 12/15
495/495 [==============================] - 309s 625